In [3]:
import rdflib

# Load the TTL file
graph = rdflib.Graph()
graph.parse("./gtc_brick.ttl", format="turtle")

# Define the SPARQL query
query = """
    PREFIX brick: <https://brickschema.org/schema/Brick#>
    
    SELECT ?chiller ?relationship ?entity
    WHERE {
    ?chiller a brick:Chiller .
    ?chiller ?relationship ?entity .
    }
"""

# Execute the SPARQL query
results = graph.query(query)

# Display the results
for result in results:
    print(f"AHU: {result.ahu}, Relationship: {result.relationship}, Related Entity: {result.relatedEntity}")


In [5]:
# Retry querying the RDF graph to extract instances of common HVAC-related classes

# Load the TTL file into an RDF graph
graph = rdflib.Graph()
graph.parse("./gtc_brick.ttl", format="turtle")

# List of common HVAC-related classes in Brick schema
hvac_classes = [
    "AHU", "VAV", "Chiller", "Boiler", "Fan", "Filter", "Coil", 
    "Thermostat", "Sensor", "Actuator", "Valve", "Pump"
]

# Namespace for Brick
brick_ns = rdflib.Namespace("https://brickschema.org/schema/Brick#")

# Dictionary to store instances of each class
hvac_instances = {}

# Querying for instances of each class
for hvac_class in hvac_classes:
    query = f"""
        SELECT ?instance
        WHERE {{
            ?instance a brick:{hvac_class} .
        }}
    """
    # Execute the query
    results = graph.query(query)
    
    # Store the instances in the dictionary
    hvac_instances[hvac_class] = [str(result[0]) for result in results]

# Display the number of instances for each HVAC class
{class_name: len(instances) for class_name, instances in hvac_instances.items()}


{'AHU': 1,
 'VAV': 12,
 'Chiller': 0,
 'Boiler': 0,
 'Fan': 2,
 'Filter': 2,
 'Coil': 0,
 'Thermostat': 0,
 'Sensor': 0,
 'Actuator': 0,
 'Valve': 1,
 'Pump': 3}

In [1]:
import rdflib

# Load the TTL file into an RDF graph
graph = rdflib.Graph()
graph.parse("./gtc_brick.ttl", format="turtle")

# Identified HVAC components
# hvac_classes = ['AHU', 'VAV', 'Fan', 'Filter', 'Valve', 'Pump']
hvac_classes = ['AHU', 'VAV']

# Namespace for Brick
brick_ns = rdflib.Namespace("https://brickschema.org/schema/Brick#")

# Initialize a graph for the subset schema
subset_graph = rdflib.Graph()

# Loop through each HVAC component type and extract relevant triples
for hvac_class in hvac_classes:
    construct_query = f"""
        CONSTRUCT {{
            ?instance ?p ?o .
            ?o ?p2 ?o2 .
        }}
        WHERE {{
            ?instance a brick:{hvac_class} .
            ?instance ?p ?o .
            OPTIONAL {{ ?o ?p2 ?o2 }}
        }}
    """
    # Execute the CONSTRUCT query and merge the result with the subset graph
    subset_graph += graph.query(construct_query)

# Save the extracted subset schema to a new TTL file
subset_graph.serialize(destination="./hvac_subset_ahu_vav.ttl", format="turtle")


<Graph identifier=N88e6c5b7c4ae4333b54b5d4310af6416 (<class 'rdflib.graph.Graph'>)>